In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from PMTK.sampling.preferences_sampler import *
from PMTK.pref.preferences import *
from PMTK.utility.utility_solver import *
from PMTK.sampling.subset_samplers import *
from PMTK.utility.model_solver import *
from PMTK.utility.subset_finder import *
from PMTK.sampling.gibbs import *
from PMTK.data.film_dataset import *
from PMTK.sampling.decider import *
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

def build_approx_theta(prf, init_theta = None):
    connivents = []
    if not init_theta:
        init_theta = [EMPTY_SET]
    theta = init_theta
    min_k = 1
    c  = get_connivent(theta, prf)
    cpt = 0
    while c:
        if not c in connivents:
            connivents.append(c)
        cit = get_candidate_iterator(c)
        skey = sorted(cit.keys())[0]
        b = False
        for k in cit:
            if b:
                break
            for i in cit[k]:
                s = set(i)
                for t in s:
                    b = b or (check_connivence_resolution(c, t) and not t in theta)
                    if not t in theta and check_connivence_resolution(c, t):
                        theta.append(t)
        c  = get_connivent(theta, prf)
        cpt = cpt + 1
        print("solved connivent: ", cpt, " with", theta)
    a = additivity(theta)
    for c_i in connivents:
        cit = get_candidate_iterator(c_i)
        for k in cit:
            if k > a:
                break
            for i in cit[k]:
                for t in i:
                    if not t in theta and check_connivence_resolution(c_i,t):
                        theta.append(t)
    
    return theta

In [22]:
n_users = 10
fd = Film_Dataset(100,n_users + 1)
prf = fd.get_preferences_items(n_users)
t_heuristic = build_approx_theta(prf, [EMPTY_SET])
t_heuristic

solved connivent:  1  with [(5,), (4,)]
solved connivent:  2  with [(5,), (4,), (20,)]
solved connivent:  3  with [(5,), (4,), (20,), (2,), (8,), (6,), (3,), (9,), (7,)]
solved connivent:  4  with [(5,), (4,), (20,), (2,), (8,), (6,), (3,), (9,), (7,), (18,), (19,)]


[(5,), (4,), (20,), (2,), (8,), (6,), (3,), (9,), (7,), (18,), (19,)]

In [23]:
t_mins = get_kernels_lex3(prf,t_heuristic)
t_mins

[[(4,), (20,), (8,), (18,)],
 [(4,), (20,), (9,), (18,)],
 [(5,), (20,), (8,), (18,)],
 [(4,), (20,), (8,), (19,)],
 [(5,), (20,), (9,), (19,)],
 [(5,), (20,), (8,), (19,)],
 [(4,), (20,), (9,), (19,)],
 [(5,), (20,), (9,), (18,)]]

In [24]:
prf

(0, 2, 4, 7, 8, 10, 13, 14, 16, 18, 20) > (0, 2, 4, 6, 8, 10, 12, 14, 16, 18) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18) > (1, 2, 5, 7, 8, 11, 13, 14, 17, 19) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18) > (0, 3, 5, 7, 9, 11, 12, 14, 17, 19) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18) > (1, 3, 5, 6, 8, 11, 13, 15, 17, 19) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20) > (0, 2, 4, 6, 8, 10, 12, 14, 16, 18) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18) > (0, 2, 5, 6, 8, 11, 12, 14, 17, 19) 
(0, 2, 4, 7, 8, 10, 12, 14, 16, 18, 20) > (0, 2, 4, 6, 8, 10, 12, 14, 16, 18) 
(0, 2, 4, 7, 8, 10, 13, 14, 16, 18, 20) > (0, 2, 4, 6, 8, 11, 12, 14, 17, 19) 
(0, 2, 4, 6, 8, 11, 12, 14, 17, 19) > (1, 2, 5, 7, 8, 11, 13, 14, 17, 19) 
(0, 2, 4, 6, 8, 11, 12, 14, 17, 19) > (0, 3, 5, 7, 9, 11, 12, 14, 17, 19) 
(0, 2, 4, 6, 8, 11, 12, 14, 17, 19) > (1, 3, 5, 6, 8, 11, 13, 15, 17, 19) 
(0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20) > (0, 2, 4, 6, 8, 11, 12, 14, 17, 19) 
(0, 2, 4, 6, 8, 11, 12, 14, 17, 19) > (0, 2, 5, 6, 8, 11, 12, 14, 17, 19) 
(0, 2